# Evaluating multiple feature recipes on the Simulation_Rerun_12_01_21 dataset

- **Model**: Gradient Boosted Trees (histogram-based)
- **Target(s)**: _coming from the recipes_
- **Features**: various (71 alternative _recipes_)
- **Results**: 
  - $r^2$ scores (by cross-validation)
  - feature importances (permutation-based, using the full dataset for training)
  - predictions (merged, by cross-validation)
- **Evaluation strategy**: cross-validation (leave one subject out)

## Libraries

In [1]:
# Standard library
import warnings
import os
from pathlib import Path


# Third party
import numpy as np
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.21", "Use the conda_python3_latest kernel!"
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn import (ensemble, metrics, preprocessing, 
                     pipeline, inspection, model_selection)

from IPython.display import display, Markdown

## Load Dataset

In [2]:
DATASET = Path("../datasets/Simulation_Rerun_12_01_21")
DATASET_CSV = DATASET / "Simulation_table.csv"
DATASET_README = DATASET / "Recipes and Thresholds.xlsx"
#VARIANT_NAME = "Test Moment Threshold"
#VARIANT_NAME = "Damage"
VARIANT_NAME = "Compression"

RESULTS_DIR =  Path("results") / DATASET.name / VARIANT_NAME

print(f"Reading {DATASET_CSV}...")
df_orig = pd.read_csv(DATASET_CSV)
df_orig.describe()

Reading ../datasets/Simulation_Rerun_12_01_21/Simulation_table.csv...


,M_Trial_Num,M_Mass,M_Mass_to_L5S1,M_sub_task_indices,M_sub_task_num,M_include_overall,M_Index,M_Sub,M_sub_task_num_overall,M_Index_overall,...,RWEO_01_02_00_00_INSOLE_RX_ML_threshF50_mm,RWEO_01_02_00_00_INSOLE_RY_AP_threshF50_mm,RWEF_03_00_00_00_INSOLE_LFORCE_threshF50_BW,RWEF_03_04_00_00_INSOLE_LX_ML_threshF50_BH,RWEF_03_04_00_00_INSOLE_LY_AP_threshF50_BH,RWEF_01_00_00_00_INSOLE_RFORCE_threshF50_BW,RWEF_01_02_00_00_INSOLE_RX_ML_threshF50_BH,RWEF_01_02_00_00_INSOLE_RY_AP_threshF50_BH,TO_Compression,TO_Damage
count,1.444342e+06,1.444342e+06,0.0,0.0,0.0,1.444342e+06,1.444342e+06,1.444342e+06,1444342.0,1.444342e+06,...,1.207234e+06,1.207234e+06,1.442756e+06,1.193679e+06,1.193679e+06,1.442756e+06,1.207234e+06,1.207234e+06,1.424272e+06,1.424272e+06
mean,7.898920e+01,9.926241e+00,NaN,NaN,NaN,8.743102e-01,3.502123e+03,5.401267e+00,0.0,1.121823e+05,...,4.881444e+01,1.323979e+02,4.309802e-01,2.789020e-02,6.610251e-02,4.421784e-01,2.686930e-02,7.280565e-02,1.263874e+03,1.766263e+00
std,1.660174e+01,5.168718e+00,NaN,NaN,NaN,3.314995e-01,2.566335e+03,2.931130e+00,0.0,6.195190e+04,...,9.433342e+00,5.418441e+01,3.308654e-01,5.220985e-03,3.077229e-02,3.416963e-01,5.268231e-03,3.001277e-02,9.102170e+02,9.680966e+02
min,3.500000e+01,5.000000e+00,NaN,NaN,NaN,0.000000e+00,1.000000e+00,1.000000e+00,0.0,0.000000e+00,...,1.149000e+01,2.432000e+01,0.000000e+00,6.326870e-03,1.569274e-02,0.000000e+00,6.227642e-03,1.318157e-02,2.810005e+02,1.526047e-06
25%,7.800000e+01,5.000000e+00,NaN,NaN,NaN,1.000000e+00,1.423000e+03,3.000000e+00,0.0,7.717525e+04,...,4.474000e+01,8.673000e+01,1.332044e-01,2.578125e-02,3.892602e-02,1.437663e-01,2.426415e-02,4.760981e-02,4.781508e+02,2.540399e-06
50%,8.400000e+01,1.000000e+01,NaN,NaN,NaN,1.000000e+00,2.973000e+03,5.000000e+00,0.0,1.196900e+05,...,5.085000e+01,1.314700e+02,4.021949e-01,2.881461e-02,6.135938e-02,4.001025e-01,2.809375e-02,7.169231e-02,9.795552e+02,9.285635e-06
75%,9.000000e+01,1.500000e+01,NaN,NaN,NaN,1.000000e+00,5.127000e+03,8.000000e+00,0.0,1.598110e+05,...,5.484000e+01,1.790300e+02,6.882456e-01,3.123035e-02,9.174005e-02,6.990125e-01,3.025281e-02,9.754377e-02,1.878973e+03,9.496359e-05
max,9.600000e+01,2.300000e+01,NaN,NaN,NaN,1.000000e+00,1.411700e+04,1.000000e+01,0.0,2.368970e+05,...,7.881000e+01,2.677800e+02,1.696594e+00,4.425281e-02,1.506798e-01,1.675757e+00,4.326744e-02,1.504382e-01,1.076618e+04,9.014191e+05


## Associate column names

In [3]:
def _get_columns_with_prefix(df, prefix):
    columns = []
    for column in df.columns:
        if column.startswith(prefix):
            columns.append(column)
    return columns
    
def get_target_names(df):
    return _get_columns_with_prefix(df, "T_")

def get_meta_names(df):
    return _get_columns_with_prefix(df, "M_")    

## Clean-up dataset

- Remove samples based on `M_include_overall`

In [4]:
df = df_orig[df_orig["M_include_overall"] > 0]

# Weed out wonky subjects
#df = df[df["M_Sub"].isin([2,4,5,6,7,8,9])]
#RESULTS_DIR += "_nowonky"

print(f"Number of samples: {df.shape[0]:,d} (before clean-up: {df_orig.shape[0]:,d})")
print(f"Number of trials: {len(df['M_Trial_Name'].unique())} (before clean-up: {len(df_orig['M_Trial_Name'].unique())})")
print(f"Number of subjects: {len(df['M_Sub'].unique())}")

Number of samples: 1,262,803 (before clean-up: 1,444,342)
Number of trials: 57 (before clean-up: 57)
Number of subjects: 10


## Predictor configurations (recipes)

In [5]:
def predictor_short_name(predictor):
    return predictor[17:]

def predictor_sensor_number(predictor):
    #return int(predictor[5:7])
    return predictor[5:7]

def filter_predictors(all_predictors, patterns):
    if isinstance(patterns, str):
        patterns = (patterns,)
        
    predictors = []
    for predictor in all_predictors:
        for pattern in patterns:
            if pattern in predictor:
                predictors.append(predictor)
                break
    return predictors


def build_feature_sets(df, readme):    
    feature_sets = {}
    
    recipes = readme.iteritems()
    next(recipes)   # first column is bogus
    for recipe_num, recipe in recipes:
        recipe_desc = recipe[3]
        recipe_filter_1 = [filter for filter in (recipe[6], recipe[7]) if isinstance(filter, str)]
        recipe_filter_2 = [filter for filter in recipe[10:16] if isinstance(filter, str)]
        recipe_name = f"Recipe {recipe_num}: {recipe_desc}"
        feature_sets[recipe_name] = filter_predictors(filter_predictors(df.columns, recipe_filter_1), recipe_filter_2)
    
    return feature_sets

readme = pd.read_excel(DATASET_README, sheet_name=VARIANT_NAME)
target_name = readme.iloc[4, 1]
target_threshold = readme.iloc[17, 1]

# Threshold filtering for everything
df = df[df[target_name] > target_threshold]

feature_sets = build_feature_sets(df, readme)

for feature_set_name, predictors in feature_sets.items():
    sensors = set(map(predictor_sensor_number, predictors))
    print(f"{feature_set_name}\n\tPredictors: {len(predictors)}, Sensors: {len(sensors)}\n")

Recipe 1: insole,trunk
	Predictors: 15, Sensors: 3

Recipe 2: trunk
	Predictors: 3, Sensors: 1



In [6]:
def evaluate(target_name, feature_names):
    X, y, groups = df[feature_names], df[target_name], df["M_Sub"]
    
    model = pipeline.Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('gboost', ensemble.HistGradientBoostingRegressor())
    ])
    
    logo = model_selection.LeaveOneGroupOut()

    prediction = model_selection.cross_val_predict(
        model, X, y, cv=logo, groups=groups, n_jobs=-1)

    r2_score = {}
    for idx_train, idx_test in logo.split(df, groups=groups):
        subject = df.iloc[idx_test[0]]["M_Sub"]
        test_target = y.iloc[idx_test]
        test_pred = prediction[idx_test]
        
        # Threshold filtering for testing, only
        threshold_mask = test_target > target_threshold
        #print("DBG Before", test_target.shape, test_pred.shape)
        test_target = test_target[threshold_mask]
        test_pred = test_pred[threshold_mask]
        #print("DBG After", test_target.shape, test_pred.shape)
        
        r2_score[subject] = metrics.r2_score(test_target, test_pred)
        
    r2_score = pd.Series(r2_score)
    prediction = pd.Series(prediction, index=y.index)
    
    # Feature importances on the full training set
    model.fit(X, y)
    perm_imp = inspection.permutation_importance(model, X, y, n_repeats=5, n_jobs=10)
    importance = pd.Series(perm_imp.importances_mean, index=X.columns)
    importance.sort_values(ascending=False, inplace=True)

    return r2_score, importance, prediction, y

## Run experiments, save data

In [7]:
os.makedirs(RESULTS_DIR, exist_ok=True)


r2_mean_scores = {}

for feature_set_name, feature_names in feature_sets.items():
    r2_score, importance, prediction, target = evaluate(target_name, feature_names)
    r2_mean_scores[feature_set_name] = r2_score.mean()
    display(
        Markdown(
            "---\n"
            f"**Features**: {feature_set_name}  \n"
            f"**$R^2$ ({target_name}) = {r2_mean_scores[feature_set_name]:.3f}**\n"
        )
    )

    with pd.ExcelWriter(f"{RESULTS_DIR}/R2_scores.xlsx") as writer:
        df_results = pd.DataFrame({f"R2 - {target_name}": r2_mean_scores,})
        df_results.to_excel(writer, sheet_name="R2 Scores")


    short_name = feature_set_name.split(":")[0].replace(" ", "_")
    with pd.ExcelWriter(f"{RESULTS_DIR}/{short_name}_results.xlsx") as writer:

        df_results = pd.DataFrame({f"R2 - {target_name}": r2_score,})
        df_results.to_excel(writer, index_label="Test Subject", sheet_name="R2 Scores")


        df_results = pd.DataFrame(
            {
                #"Short name": map(predictor_short_name, importance_x.index),
                f"Importance - {target_name}": importance,
            }
        )
        df_results.to_excel(writer, sheet_name="Importance")
        
    df_results = pd.DataFrame(
        {
            f"Predictions - {target_name}": prediction,
            f"Target - {target_name}": target,
        }
    )
    df_results.to_csv(f"{RESULTS_DIR}/{short_name}_predictions.csv")



---
**Features**: Recipe 1: insole,trunk  
**$R^2$ (TO_Compression) = 0.359**


---
**Features**: Recipe 2: trunk  
**$R^2$ (TO_Compression) = -0.817**
